In [1]:
import pandas as pd
import numpy as np

from core.bot import get_net_strength_series
from utils.net_sma_trend import get_net_trend

In [2]:
from api.constants_test_1 import ACCOUNT_ID, API_KEY, OANDA_URL
from api.OandaApi import OandaApi
from utils.heiken_ashi import ohlc_to_heiken_ashi
import matplotlib.pyplot as plt

def no_op(*args, **kwargs) -> None:
    pass

In [3]:
api = OandaApi(ACCOUNT_ID, API_KEY, OANDA_URL)

In [4]:
candles = api.get_candles_df("USD_JPY", count=1000, granularity="M15", completed_only=True)
candles = candles[["time", "mid_c", "mid_o", "mid_h", "mid_l"]].copy()
# candles = candles.iloc[0:83].copy()
heiken_ashi = ohlc_to_heiken_ashi(candles)

heiken_ashi.tail(5)

,time,ha_open,ha_high,ha_low,ha_close,ha_green,ha_streak,ha_open_at_extreme
994,2025-06-17 19:00:00+00:00,145.276649,145.376000,145.276649,145.34750,1,6,1
995,2025-06-17 19:15:00+00:00,145.312075,145.381000,145.298000,145.33675,1,7,0
996,2025-06-17 19:30:00+00:00,145.324412,145.358000,145.253000,145.29500,0,-1,0
997,2025-06-17 19:45:00+00:00,145.309706,145.309706,145.207000,145.24100,0,-2,1
998,2025-06-17 20:00:00+00:00,145.275353,145.291000,145.205000,145.25000,0,-3,0


In [5]:
candles.tail(5)

,time,mid_c,mid_o,mid_h,mid_l
994,2025-06-17 19:00:00+00:00,145.357,145.369,145.376,145.288
995,2025-06-17 19:15:00+00:00,145.310,145.358,145.381,145.298
996,2025-06-17 19:30:00+00:00,145.260,145.309,145.358,145.253
997,2025-06-17 19:45:00+00:00,145.218,145.261,145.278,145.207
998,2025-06-17 20:00:00+00:00,145.288,145.216,145.291,145.205


In [6]:
merged_df = pd.merge(candles, heiken_ashi, on="time", how="inner", suffixes=("", "_ha"))

In [7]:
merged_df.tail(5)

,time,mid_c,mid_o,mid_h,mid_l,ha_open,ha_high,ha_low,ha_close,ha_green,ha_streak,ha_open_at_extreme
994,2025-06-17 19:00:00+00:00,145.357,145.369,145.376,145.288,145.276649,145.376000,145.276649,145.34750,1,6,1
995,2025-06-17 19:15:00+00:00,145.310,145.358,145.381,145.298,145.312075,145.381000,145.298000,145.33675,1,7,0
996,2025-06-17 19:30:00+00:00,145.260,145.309,145.358,145.253,145.324412,145.358000,145.253000,145.29500,0,-1,0
997,2025-06-17 19:45:00+00:00,145.218,145.261,145.278,145.207,145.309706,145.309706,145.207000,145.24100,0,-2,1
998,2025-06-17 20:00:00+00:00,145.288,145.216,145.291,145.205,145.275353,145.291000,145.205000,145.25000,0,-3,0


In [8]:
combined_df = get_net_strength_series(merged_df, no_op)

In [9]:
combined_df.columns

Index(['time', 'mid_c', 'mid_o', 'mid_h', 'mid_l', 'ha_open', 'ha_high',
       'ha_low', 'ha_close', 'ha_green', 'ha_streak', 'ha_open_at_extreme',
       'trix_9', 'trix_15', 'sma_200', 'sma_100', 'sma_50', 'sma_30', 'sma_10',
       'net_trend_200', 'net_trend_100', 'net_trend_50', 'net_trend_30',
       'bearish_strength', 'bullish_strength', 'bearish_strength_s',
       'bullish_strength_s', 'net_strength', 'net_strength_s'],
      dtype='object')

In [10]:
combined_df = combined_df[['time', 'mid_c', 'mid_o', 'mid_h', 'mid_l', 'ha_open', 'ha_high',
       'ha_low', 'ha_close', 'ha_green', 'ha_streak', 'ha_open_at_extreme',
       'net_strength_s']].copy()

In [11]:
candles_high = api.get_candles_df("USD_JPY", count=1000, granularity="H1", completed_only=True)
candles_high = candles_high[["time", "mid_c", "mid_o", "mid_h", "mid_l"]].copy()

combined_df_high = get_net_strength_series(candles_high, no_op)


combined_df_high = combined_df_high[['time', 'mid_c', 'mid_o', 'mid_h', 'mid_l', 'net_strength_s']]










In [12]:
combined_df_high.tail(5)

,time,mid_c,mid_o,mid_h,mid_l,net_strength_s
994,2025-06-17 15:00:00+00:00,145.040,145.185,145.208,145.012,1.25
995,2025-06-17 16:00:00+00:00,145.224,145.039,145.258,144.908,1.29
996,2025-06-17 17:00:00+00:00,145.248,145.223,145.303,145.130,1.38
997,2025-06-17 18:00:00+00:00,145.368,145.246,145.378,145.191,1.45
998,2025-06-17 19:00:00+00:00,145.218,145.369,145.381,145.207,1.51


In [13]:
combined_df_high["time"] = pd.to_datetime(combined_df_high["time"]).dt.tz_convert(None)
combined_df["time"] = pd.to_datetime(combined_df["time"]).dt.tz_convert(None)

In [14]:
def granularity_to_freq(granularity):
    mapping = {
        "M1": "T",  # 1-minute granularity
        "M5": "5T",  # 5-minute granularity
        "M15": "15T",  # 15-minute granularity
        "M30": "30T",  # 30-minute granularity
        "H1": "h",  # Hourly granularity
        "H4": "4h",  # 4-hour granularity
        "D": "D"  # Daily granularity
    }
    return mapping.get(granularity, "D")  # Default to 'D' if granularity not in mapping

long_freq = granularity_to_freq("H1")

In [15]:
long_freq

'h'

In [16]:
combined_df_high["date"] = combined_df_high["time"].dt.floor(long_freq)
combined_df["date"] = combined_df["time"].dt.floor(long_freq)
combined_df

,time,mid_c,mid_o,mid_h,mid_l,ha_open,ha_high,ha_low,ha_close,ha_green,ha_streak,ha_open_at_extreme,net_strength_s,date
0,2025-06-03 10:30:00,142.925,142.988,143.040,142.902,142.988000,143.040000,142.902000,142.96375,0,-1,0,0.00,2025-06-03 10:00:00
1,2025-06-03 10:45:00,143.059,142.924,143.083,142.918,142.975875,143.083000,142.918000,142.99600,1,1,0,0.00,2025-06-03 10:00:00
2,2025-06-03 11:00:00,143.101,143.060,143.132,143.048,142.985938,143.132000,142.985938,143.08525,1,2,1,0.04,2025-06-03 11:00:00
3,2025-06-03 11:15:00,143.149,143.100,143.164,143.100,143.035594,143.164000,143.035594,143.12825,1,3,1,0.07,2025-06-03 11:00:00
4,2025-06-03 11:30:00,143.114,143.152,143.156,143.067,143.081922,143.156000,143.067000,143.12225,1,4,0,0.09,2025-06-03 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2025-06-17 19:00:00,145.357,145.369,145.376,145.288,145.276649,145.376000,145.276649,145.34750,1,6,1,1.72,2025-06-17 19:00:00
995,2025-06-17 19:15:00,145.310,145.358,145.381,145.298,145.312075,145.381000,145.298000,145.33675,1,7,0,1.70,2025-06-17 19:00:00
996,2025-06-17 19:30:00,145.260,145.309,145.358,145.253,145.324412,145.358000,145.253000,145.29500,0,-1,0,1.68,2025-06-17 19:00:00
997,2025-06-17 19:45:00,145.218,145.261,145.278,145.207,145.309706,145.309706,145.207000,145.24100,0,-2,1,1.62,2025-06-17 19:00:00


In [17]:
combined_df_high

,time,mid_c,mid_o,mid_h,mid_l,net_strength_s,date
0,2025-04-21 05:00:00,140.646,140.768,140.788,140.621,0.00,2025-04-21 05:00:00
1,2025-04-21 06:00:00,140.762,140.646,140.910,140.621,0.00,2025-04-21 06:00:00
2,2025-04-21 07:00:00,140.555,140.763,140.824,140.546,-0.04,2025-04-21 07:00:00
3,2025-04-21 08:00:00,140.576,140.556,140.730,140.548,-0.07,2025-04-21 08:00:00
4,2025-04-21 09:00:00,140.658,140.576,140.695,140.471,-0.09,2025-04-21 09:00:00
...,...,...,...,...,...,...,...
994,2025-06-17 15:00:00,145.040,145.185,145.208,145.012,1.25,2025-06-17 15:00:00
995,2025-06-17 16:00:00,145.224,145.039,145.258,144.908,1.29,2025-06-17 16:00:00
996,2025-06-17 17:00:00,145.248,145.223,145.303,145.130,1.38,2025-06-17 17:00:00
997,2025-06-17 18:00:00,145.368,145.246,145.378,145.191,1.45,2025-06-17 18:00:00


In [18]:
merged_df = pd.merge(combined_df, combined_df_high, on="date", how="inner", suffixes=("", "_long"))

In [19]:
merged_df

,time,mid_c,mid_o,mid_h,mid_l,ha_open,ha_high,ha_low,ha_close,ha_green,ha_streak,ha_open_at_extreme,net_strength_s,date,time_long,mid_c_long,mid_o_long,mid_h_long,mid_l_long,net_strength_s_long
0,2025-06-03 10:30:00,142.925,142.988,143.040,142.902,142.988000,143.040000,142.902000,142.96375,0,-1,0,0.00,2025-06-03 10:00:00,2025-06-03 10:00:00,143.059,142.858,143.083,142.838,-1.33
1,2025-06-03 10:45:00,143.059,142.924,143.083,142.918,142.975875,143.083000,142.918000,142.99600,1,1,0,0.00,2025-06-03 10:00:00,2025-06-03 10:00:00,143.059,142.858,143.083,142.838,-1.33
2,2025-06-03 11:00:00,143.101,143.060,143.132,143.048,142.985938,143.132000,142.985938,143.08525,1,2,1,0.04,2025-06-03 11:00:00,2025-06-03 11:00:00,143.084,143.060,143.164,143.048,-1.28
3,2025-06-03 11:15:00,143.149,143.100,143.164,143.100,143.035594,143.164000,143.035594,143.12825,1,3,1,0.07,2025-06-03 11:00:00,2025-06-03 11:00:00,143.084,143.060,143.164,143.048,-1.28
4,2025-06-03 11:30:00,143.114,143.152,143.156,143.067,143.081922,143.156000,143.067000,143.12225,1,4,0,0.09,2025-06-03 11:00:00,2025-06-03 11:00:00,143.084,143.060,143.164,143.048,-1.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,2025-06-17 18:45:00,145.368,145.241,145.378,145.241,145.246299,145.378000,145.241000,145.30700,1,5,0,1.71,2025-06-17 18:00:00,2025-06-17 18:00:00,145.368,145.246,145.378,145.191,1.45
994,2025-06-17 19:00:00,145.357,145.369,145.376,145.288,145.276649,145.376000,145.276649,145.34750,1,6,1,1.72,2025-06-17 19:00:00,2025-06-17 19:00:00,145.218,145.369,145.381,145.207,1.51
995,2025-06-17 19:15:00,145.310,145.358,145.381,145.298,145.312075,145.381000,145.298000,145.33675,1,7,0,1.70,2025-06-17 19:00:00,2025-06-17 19:00:00,145.218,145.369,145.381,145.207,1.51
996,2025-06-17 19:30:00,145.260,145.309,145.358,145.253,145.324412,145.358000,145.253000,145.29500,0,-1,0,1.68,2025-06-17 19:00:00,2025-06-17 19:00:00,145.218,145.369,145.381,145.207,1.51
